In [11]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

In [12]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [34]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if label_visiprog[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label

print(np.max(label_visiprog))

202.0


In [31]:
accuracy = []
accuracy_vsp = []

N_material = 61
N_material_train = 5

for fold in range(10):
    try:
        print("Trial {}".format(fold))

        idx_train = np.array([])
        for i in range(N_material_train):
            train_class = np.random.randint(0, N_material)
            idx_train= np.concatenate((idx_train, np.where(label_material == train_class)[0]))

        # need to get better code later
        idx_test = np.setdiff1d(np.arange(N), idx_train)

        idx_train = idx_train.astype(int)
        idx_test = idx_test.astype(int)

        X_train = raw_feature[idx_train,:]
        Y_VSP_train = label_visiprog[idx_train]
        Y_mat_train = label_material[idx_train]

        X_test = raw_feature[idx_test,:]
        Y_VSP_test = label_visiprog[idx_test]
        Y_mat_test = label_material[idx_test]

        model_vsp = LFDA_VISIPROG()
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])


        model = LFDA()
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = leave_one_sample_out(X_mat_test, Y_mat_test)
        accuracy.append(res['accuracy'])
    
    except:
        print("Skipping trial {}".format(fold))

/Users/andrey/Dropbox/Hacking/Research/VisiProg2/analysis/thesis/visiprog/metric_learn/fda_visiprog.py:129: RuntimeWarning: invalid value encountered in sqrt
  vecs *= np.sqrt(vals)


Skipping 4 trial
Skipping 7 trial
Skipping 9 trial


In [32]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy})
display(df)

,material,visiprog
0,0.932497,0.924347
1,0.925174,0.884789
2,0.927064,0.912226
3,0.887147,0.891327
4,0.937257,0.922288
5,0.926799,0.910601
6,0.926327,0.889906


In [33]:
df.mean()

material    0.923181
visiprog    0.905069
dtype: float64

In [ ]:
print(model.transformer())